In [1]:
!uv pip install sagemaker
!uv pip install boto3

Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 1.25s
Using Python 3.10.15 environment at C:\Users\bored\Music\SagemekerLearning\sagelearner
Audited 1 package in 8ms


In [2]:
job_name = 'iris-xgboost-2025-06-08-17-43-29'


In [3]:
from sagemaker.estimator import Estimator

xgb_model = Estimator.attach(job_name)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml

2025-06-08 12:16:32 Starting - Preparing the instances for training
2025-06-08 12:16:32 Downloading - Downloading the training image
2025-06-08 12:16:32 Training - Training image download completed. Training in progress.
2025-06-08 12:16:32 Uploading - Uploading generated training model
2025-06-08 12:16:32 Completed - Training job completed


In [4]:
from sagemaker.serializers import CSVSerializer

In [7]:
xgb_model_predictor = xgb_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    serializer=CSVSerializer()
)

-----------!

In [8]:
xgb_model_predictor.predict(
    '5.1,3.5,1.4,0.2'
)

b'0.0'

In [9]:
endpoint_name = xgb_model_predictor.endpoint_name

In [12]:
import boto3

In [13]:
sagamaker_runtime = boto3.client('sagemaker-runtime')

In [14]:
payload = '5.1,3.5,1.4,0.2'
response = sagamaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=payload
)

In [15]:
response_body = response['Body'].read().decode('utf-8')
print(response_body)


0.0


In [16]:
payload = '5.1,3.5,1.4,0.2 \n 5.9,3.0,5.1,1.8'
response = sagamaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=payload
)

In [17]:
response_body = response['Body'].read().decode('utf-8')
print(response_body)

0.0,2.0


In [19]:
csv_buffer=open('data/infer.csv')

In [24]:
import pandas as pd
infer_df = pd.read_csv("data/infer.csv", header=None)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 import pandas as pd                                                                          │
│ ❱ 2 infer_df = pd.read_csv("data/infer.csv", header=None)                                        │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\pandas\io\parsers\readers.p │
│ y:1026 in read_csv                                                                               │
│                                                                                                  │
│   1023 │   )                                                                                     │
│   1024 │   kwds.update(kwds_defaults)                                                            │
│   1025 │                                                                                         │
│ ❱ 1026 │   return _read(filepath_or_buffer, kwds)                                                │
│   1027                                                                                           │
│   1028                                                                                           │
│   1029 # iterator=True -> TextFileReader                                                         │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\pandas\io\parsers\readers.p │
│ y:620 in _read                                                                                   │
│                                                                                                  │
│    617 │   _validate_names(kwds.get("names", None))                                              │
│    618 │                                                                                         │
│    619 │   # Create the parser.                                                                  │
│ ❱  620 │   parser = TextFileReader(filepath_or_buffer, **kwds)                                   │
│    621 │                                                                                         │
│    622 │   if chunksize or iterator:                                                             │
│    623 │   │   return parser                                                                     │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\pandas\io\parsers\readers.p │
│ y:1620 in __init__                                                                               │
│                                                                                                  │
│   1617 │   │   │   self.options["has_index_names"] = kwds["has_index_names"]                     │
│   1618 │   │                                                                                     │
│   1619 │   │   self.handles: IOHandles | None = None                                             │
│ ❱ 1620 │   │   self._engine = self._make_engine(f, self.engine)                                  │
│   1621 │                                                                                         │
│   1622 │   def close(self) -> None:                                                              │
│   1623 │   │   if self.handles is not None:                                                      │
│                                                            

In [25]:
import os
os.path.exists("data/infer.csv")


True

In [ ]:
with open("data/iris_infer.csv", 'r') as f:
    content = f.read()
    print(content)


In [29]:
csv_buffer = open("data/iris_infer.csv")

In [30]:
payload_csv_text=csv_buffer.read()
response = sagamaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=payload_csv_text
)

In [31]:
response_body = response['Body'].read().decode('utf-8')
print(response_body)

2.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


In [32]:
bucket = 'sgmkr-learner'

In [ ]:
prefix="iris"


In [36]:
import io

In [38]:
infer_ip_s3_uri = "s3://{}/{}/{}".format(
    bucket, prefix, "batch_transform/iris_infer.csv"
)

# payload_df = pd.read_csv(infer_ip_s3_uri)
# payload_df = wr.s3.read_csv(path=infer_ip_s3_uri)
s3_clnt = boto3.client("s3")
obj = s3_clnt.get_object(Bucket=bucket, Key="iris/batch_transform/iris_infer.csv")
payload_df = pd.read_csv(obj["Body"])

csv_buffer = io.StringIO()
payload_df.to_csv(csv_buffer, header=None, index=None)
payload_csv_text = payload_df.getvalue()

response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10                                                                                             │
│   11 csv_buffer = io.StringIO()                                                                  │
│   12 payload_df.to_csv(csv_buffer, header=None, index=None)                                      │
│ ❱ 13 payload_csv_text = payload_df.getvalue()                                                    │
│   14                                                                                             │
│   15 response = sgmkr_runtime.invoke_endpoint(                                                   │
│   16 │   EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text               │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\pandas\core\generic.py:6318 │
│ in __getattr__                                                                                   │
│                                                                                                  │
│    6315 │   │   │   and self._info_axis._can_hold_identifiers_and_holds_name(name)               │
│    6316 │   │   ):                                                                               │
│    6317 │   │   │   return self[name]                                                            │
│ ❱  6318 │   │   return object.__getattribute__(self, name)                                       │
│    6319 │                                                                                        │
│    6320 │   @final                                                                               │
│    6321 │   def __setattr__(self, name: str, value) -> None:                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataFrame' object has no attribute 'getvalue'

In [40]:
import boto3
import pandas as pd
import io

# S3 access
s3_clnt = boto3.client("s3")
obj = s3_clnt.get_object(Bucket=bucket, Key="iris/batch_transform/iris_infer.csv")

# Read CSV from S3 object
payload_df = pd.read_csv(obj["Body"])

# Convert dataframe to CSV string
csv_buffer = io.StringIO()
payload_df.to_csv(csv_buffer, header=None, index=None)
payload_csv_text = csv_buffer.getvalue()

# Invoke endpoint
response = sagamaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType="text/csv", 
    Body=payload_csv_text
)

print(response["Body"].read().decode())


1.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,2.0


In [41]:
sgmkr_clnt = boto3.client("sagemaker")

In [42]:
sgmkr_clnt.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '68732014-9f53-4102-9f10-f5df3c983f05',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '68732014-9f53-4102-9f10-f5df3c983f05',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 09 Jun 2025 02:18:46 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}